### Archiveception [Forensics]
1. Untar start_layer.gz archive yields two files
2. Bzip2
3. 7z yields password protected 7z archive and the password in a plaintext file
4. 7z extracts the flag.txt file  
`uscg_open{d0wNw4rd_15_7h3_0n1y_w4y_f0rw4rD}`

### Metadata [Forensics]
Strings yields the string comment with the flag in plaintext.  
`flag{334_shards_of_hail_fall_ominously}`

### Word [Forensics]
String is hidden in error image as metadata tag.  
`USCG{parliamentary_carpet}`

### Viginere [Crypto]
Just reversing the Vigneere encryption and running it in reverse 1000 times.

`uscg{morerepetitionsdoesntmeansafer}`

In [ ]:
def decrypt(key, ciphertext):
    plaintext = [chr(((ord(c)-ord(k))%26)+97) for c,k in zip(ciphertext,key)]
    return "".join(plaintext)

key = "raaojpkaufmsrez"
flag = "tmjuefrwtvjirfx"
for i in range(1000):
    key = decrypt(flag, key)
    flag = decrypt(key, flag)
print(key)
print(flag)

AAAA
1094795585

(487, 105)
(491, 119)
(461, 338)


dict_keys([]) 1
359 314

dict_keys([359]) 359
487 105

dict_keys([359, 487]) 174833
491 119

dict_keys([359, 487, 491]) 85843003
461 338

dict_keys([359, 487, 491, 461]) 39573624383

In [ ]:

parts = {}
with open("parts_test.txt", "r") as f:
    for line in f.readlines():
        p,pp = line.strip()[1:-1].split(",")
        parts[int(p)] = int(pp.strip())

with open("primes", "r") as f:
    primes = [int(p.strip()) for p in f.readlines()]

unused_primes = [p for p in primes if p not in parts.keys()]

print(f"Primes: {len(unused_primes)}/{len(primes)} primes needs to be searched")


Start with the list of primes and their corresponding modulo remainders. Let's assume you have 'n' primes and 'n' modulo remainders.
primes = [p1, p2, p3, ..., pn]
remainders = [r1, r2, r3, ..., rn]

Calculate the product of all the primes. Let's call this product 'N'.
N = p1 * p2 * p3 * ... * pn

For each prime 'pi' and its corresponding remainder 'ri', calculate the value of 'Ni', which is equal to 'N' divided by 'pi'. You can use the following formula:

Ni = N / pi

For each 'Ni', calculate the modular inverse 'Mi', which is the multiplicative inverse of 'Ni' modulo 'pi'. You can use any method to compute the modular inverse, such as the Extended Euclidean Algorithm.

Mi = Ni^(-1) mod pi

Calculate the final value 'x' by summing up the products of 'ri', 'Ni', and 'Mi' for each prime. Use the following formula:

x = (r1 * N1 * M1 + r2 * N2 * M2 + r3 * N3 * M3 + ... + rn * Nn * Mn) mod N

The resulting 'x' is the original value that corresponds to the given list of primes and modulo remainders.

Note: The CRT assumes that the primes in the list are pairwise coprime (i.e., they have no common factors). If the list of primes you have provided satisfies this condition, the CRT can be applied. If not, the conversion may not be possible.

Please let me know if you need further clarification or if you have additional questions!

In [ ]:
import numpy as np
import math
from Crypto.Util.number import long_to_bytes, bytes_to_long, getPrime

rems = [r for p,r in parts.items()]
rems.append(0)
primes = [p for p,r in parts.items()]
primes.append(0)
for missing_prime in unused_primes[65:80]:
    primes.pop()
    primes.append(missing_prime)
    N = math.prod(primes)
    for rem in range(512):
        rems.pop()
        rems.append(rem)
        Ni = [int(N/prime) for prime in primes]
        try:
            Mi = [pow(n,-1,p) for n,p in zip(Ni,primes)]
            x = np.sum([r*n*m] for r,n,m in zip(rems,Ni,Mi)) % N
            print( missing_prime, rem, long_to_bytes(int(x)) )
        except:
            continue
    

### The First Digram Substitution Cipher [Crypto]
The cipher is a Digram Substitution Cipher, the playfair cipher.  Bruteforcing is easy for the little keyspace.
Generating all keys, while doing some backtracking when encountering errors seemed to the trick.  Yielding a subset of the plaintext set, which was searchable.

`uscg{THIS_CIPHER_WAS_ONCE_REJECTED_FOR_ITS_PERCEIVED_COMPLEXITY}`

In [ ]:
import string
from pycipher import Playfair

secret = "SRJEEUJLTCDWWEIETCSINEOSFGTLMEOJTCENJUWNLEPICOUKFR".lower()
KEY = list("CH_R_ESWTO_BD__IJ__P_____".lower())
REM = [val for val in string.ascii_lowercase if val not in KEY and val != "q"]

def rec(key, alph):
    if "_" not in key:
        decoded = Playfair("".join(key)).decipher(secret)
        if decoded.startswith("THISCIPHERWASO"):
            print("".join(key), decoded)
            return 0
        return 5
    mod_idx = key.index("_")
    for val in alph:
        if val in key:
            continue
        key[mod_idx] = val
        ret = rec(key, alph)
        key[mod_idx] = "_"
        if ret != 0:
            return ret-1
    return 0
rec(KEY, REM)

### Table Tennis
The PCAP file contains a Ping Pong session.  However, the "Window Size Scaling factor" appears to be harbouring regular ASCII data.  
These packets are only sent from the user with the `ping` messages, and words like `ls`, `base64` and `flag` appears.

Extracting all packages to text format with all contents, We can filter out all rows, not starting with the `00 30` byte index, and continue fiddling down the text until we get something like below which, when concatenating the two first chars of each line becomes, `ls flag base64 dXNjZ3tMMG9rMW5HX3RocjB1Z0hfVGgzX3cxbkRvV30=`, and translates into `uscg{L0ok1nG_thr0ugH_Th3_w1nDoW}`

```
lsW..........8..

fl]..........9..
agb..........9..

baa..........:..
sePr.........:..
64...........;..

dX_^.........<..
NjuB.........<..
Z3in.........=..
tMOI.........=..
MGvE.........=..
9r...........=..
MWv..........>..
5H.$.........>..
X3k..........?..
Rop..........?..
cj_..........?..
B1...........@..
Z0i..........@..
hfZ..........@..
VGl..........A..
gz[..........A..
X3j..........A..
cx_..........B..
bk`..........B..
Rvpu.........C..
V3l..........C..
0=...........C..
```




`qMtStnD3Q-1e3-yYXCKOHcPagm-qiGkcnJcDuzOoac8`

### Semaphore
The conversation contains a bunch of erronuous TCP packets from A to B.  
Looking at the packets, we see the TCP flags appears to be readable, ASCII values.  
Exporting the entire conversation to JSON, then filtering out the TCP flag values, and grabbing the last byte of each we get a sequence of byte values, which converts into ASCII characters.

After converting, concatenating and decoding the bytearray using Base64, we get a text with the flag `USCG{s3map4h0r3_tcp_f1ag5}`

In [ ]:
import json
import base64
with open("semaphore.json") as jsonfile:
    data = json.load(jsonfile)
content = bytearray()
for packet in data:
    tcp_flag_str = packet["_source"]["layers"]["tcp"]["tcp.flags"][-2:]
    tcp_flag_int = int(tcp_flag_str, 16)
    tcp_flag_byte = bytes.fromhex(tcp_flag_str)
    content.append(tcp_flag_int)
print(content)
print()
print(base64.b64decode(content))

### Rumble In The Jumble

The file is encoded using a substitution cipher for the key space `[A-Za-z]`.  By analysing a standard ELF file we can generate a partial translation table which is shown below.  
```
Key space: abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
Translate: pbrisgm.ydfaeohvxl.c..tnzuF.CGPJ.R.BM.ES.TLNDU.QI...
```

In the file we find the flag string which needs to be deciphered.
```
00002000: 0100 0200 0000 0000 446e 6e6a 2046 6e62  ........Dnnj Fnb
00002010: 2120 506f 6d20 6b72 6c66 2064 653a 2054  ! Pom krlf de: T
00002020: 4e43 447b 6b31 7133 6a5f 776f 335f 757a  NCD{k1q3j_wo3_uz
00002030: 6738 7233 357d 0000 011b 033b 2c00 0000  g8r35}.....;,...
```

Using the transposition table we get
```
Dnnj Fnb! Pom krlf de: TNCD{k1q3j_wo3_uzg8r35}
Good Job! The flag is: USCG{f1x3d_th3_jum8l35}
```


### Door Dilemma 0

In [ ]:
from pwn import remote

con = remote("0.cloud.chals.io", 31598)

dataset = []
for round_idx in range(1,37):
    values = ["","",""]
    print(f"Round: {round_idx}")
    counter = 0
    while "" in values:
        r = con.recvuntil(b"Choose a door", timeout=1)
        if counter == 0:
            con.sendline(b"1")
        else:
            idx = values.index("")
            print(f"EMPTY: {idx}")
            if idx == 1:
                con.sendline(b"2")
            elif idx == 2:
                con.sendline(b"3")
        resp = con.recvline_startswith(b"Before we open it", timeout=1).decode().strip()
        print(resp)
        i = int(resp[resp.rfind("#")+1])-1
        v = resp[resp.rfind("'")-1]
        print(resp)
        print(i,v)
        if v not in values:
            values[i]=v
        
        r = con.recv(numb=1024, timeout=1)
        con.sendline(b"0")
        resp = con.recvline().decode().strip()
        print(resp)
        i = int(resp[resp.rfind("#")+1])-1
        v = resp[resp.rfind("'")-1]
        print(i,v)
        if v not in values:
            values[i]=v
        
        r = con.recvline_startswith(b"Do you want to replay this round", timeout=1)
        if "" in values:
            con.sendline(b"0")
            print(f"\tReplay {round_idx}: {values}")
        else:
            dataset.append(values)
            con.sendline(b"1")
            print(f"\tContinue to {round_idx+1}: {values}")
        counter += 1

for idx,vals in enumerate(dataset):
    print(f"{idx+1}: {vals}")

In [ ]:
from pwn import remote
con = remote("0.cloud.chals.io",30647)

names = ["Gengar", "Typhlosion", "MissingNo"]
filenames = ["flag.txt", "Typhlosion.txt", "Gengar.txt", "MissingNo.txt"]

while resp := con.recvline(timeout=1):
    print(resp.decode())
    if resp.startswith("Would you like to play?"):
        con.sendline(b"Yes")
    elif resp.startswith("?"):

In [15]:
with open("mspaint.DMP", "rb") as dmp:
    memory = dmp.read()

fsig = b"\x89\x50\x4e\x47\x0d\x0a\x1a\x0a"
fsig_len = 8
fend = b"\x49\x45\x4E\x44"
fend_len = 4

fsig = b"\xFF\xD8"
fsig_len = 2
fsig = b"\xFF\xD9"
fend_len = 2

file_index=0
idx = 0
while idx < len(memory):
    idx = memory.find(fsig, idx)
    if idx != -1:
        offset = memory.find(fend, idx)
        if offset != -1:
            with open(f"dump/file_{file_index}.jpg", "wb") as sv:
                sv.write(memory[idx:offset])
                file_index += 1
            idx += offset
            continue
    break